In [13]:
from sympy import *
import numpy as np 
init_printing()

comp = symbols( 'comp' )

# Definitions
eta, dx, dz = symbols( 'eta, dx, dz' )
D11, D12, D13, D21, D22, D23 ,D31, D32, D33 = symbols('D11, D12, D13, D21, D22, D23 ,D31, D32, D33')
Exx, Ezz, Gxz = symbols('Exx, Ezz, Gxz')
stress_labels = ['Txx', 'Tzz', 'Txz']

D11,D12,D13,D21,D22,D23,D31,D32,D33 = symbols( 'D11,D12,D13,D21,D22,D23,D31,D32,D33' )


Dv = Matrix( [ [D11, D12, D13], [D21, D22, D23], [D31, D32, D33]  ] )
Ed = Matrix( [ [Exx], [Ezz], [Gxz] ] )
T  = Dv*Ed;
for i in range(3):
    print(stress_labels[i] + ' = ' + ccode( T[i]) + ';' )

Txx = D11*Exx + D12*Ezz + D13*Gxz;
Tzz = D21*Exx + D22*Ezz + D23*Gxz;
Txz = D31*Exx + D32*Ezz + D33*Gxz;


In [16]:
# Compressibilitz switch
comp = symbols('comp')

# Stencil x
uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE = symbols('uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE')

dofs = Matrix([uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE])

# Stencil extension
uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE = symbols('uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE')

dofs_ext = Matrix([uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE])
dofs = np.append(dofs,dofs_ext)

inW,inE,inSu,inSv,inNu,inNv = symbols('inW,inE,inSu,inSv,inNu,inNv')

inSE,inSW,inNE,inNW,inNEu,inNWu,inSEu,inSWu, inNEv,inNWv,inSEv,inSWv = symbols('inSE,inSW,inNE,inNW,inNEu,inNWu,inSEu,inSWu, inNEv,inNWv,inSEv,inSWv')

# Divergences
divW      = (uC  - uW )/dx + (vNW - vSW )/dz
divE      = (uE  - uC )/dx + (vNE - vSE )/dz
divSW     = (uS  - uSW)/dx + (vSW - vSSW)/dz
divSE     = (uSE - uS )/dx + (vSE - vSSE)/dz
divNW     = (uN  -uNW )/dx + (vNNW- vNW )/dz
divNE     = (uNE -uN  )/dx + (vNNE- vNE )/dz
# Deviatoric normal strain
ExxW      = inW*((uC -uW )/dx - comp*1/3*divW) 
EzzW      = inW*((vNW-vSW)/dz - comp*1/3*divW)
ExxE      = inE*((uE -uC )/dx - comp*1/3*divE)
EzzE      = inE*((vNE-vSE)/dz - comp*1/3*divE)
# Shear strain
GxzN      = inNu*(uN - uC )/dz + inNv*(vNE -vNW )/dx 
GxzS      = inSu*(uC - uS )/dz + inSv*(vSE -vSW )/dx
# Additional missing stress tensor components via interpolation
GxzNE     = inNEu*(uNE - uE )/dz + inNEv*( vNEE - vNE  )/dx
GxzSE     = inSEu*(uE  - uSE)/dz + inSEv*( vSEE - vSE  )/dx
GxzNW     = inNWu*(uNW - uW )/dz + inNWv*( vNW  - vNWW )/dx
GxzSW     = inSWu*(uW  - uSW)/dz + inSWv*( vSW  - vSWW )/dx
ExxNE     = inNE*((uNE -uN )/dx - comp*1/3*divNE)
ExxNW     = inNW*((uN  -uNW)/dx - comp*1/3*divNW)
ExxSE     = inSE*((uSE -uS )/dx - comp*1/3*divSE)
ExxSW     = inSW*((uS  -uSW)/dx - comp*1/3*divSW)
EzzNE     = inNE*((vNNE-vNE)/dz - comp*1/3*divNE)
EzzNW     = inNW*((vNNW-vNW)/dz - comp*1/3*divNW)
EzzSE     = inSE*((vSE-vSSE)/dz - comp*1/3*divSE)
EzzSW     = inSW*((vSW-vSSW)/dz - comp*1/3*divSW)
# INV 0
GxzE      = 1/4*( GxzN + GxzS + GxzNE + GxzSE)
GxzW      = 1/4*( GxzN + GxzS + GxzNW + GxzSW)
ExxN      = 1/4*( ExxE + ExxW + ExxNW + ExxNE)
ExxS      = 1/4*( ExxE + ExxW + ExxSW + ExxSE)
EzzN      = 1/4*( EzzE + EzzW + EzzNW + EzzNE) 
EzzS      = 1/4*( EzzE + EzzW + EzzSW + EzzSE)


#----------------------------------------#
# ExxW = 1/dx*( uC- uW) - comp*1/3*divW
# EzzW = 1/dz*(vNW-vSW) - comp*1/3*divW
# GxzW = 1/4*(GxzS+GxzN+GxzSW+GxzNW)
EdW  = Matrix([[ExxW], [EzzW], [GxzW]])
TW   = Dv*EdW
TxxW = TW[0].subs({D11:'D11W', D12:'D12W', D13:'D13W'})
#----------------------------------------#
# ExxE = 1/dx*( uE- uC) - comp*1/3*divE
# EzzE = 1/dz*(vNE-vSE) - comp*1/3*divE
# GxzE = 1/4*(GxzS+GxzN+GxzSE+GxzNE)
EdE  = Matrix([[ExxE], [EzzE], [GxzE]])
TE   = Dv*EdE
TxxE = TE[0].subs({D11:'D11E', D12:'D12E', D13:'D13E'})
#----------------------------------------#
# ExxS = 1/4*(ExxE+ExxW+ExxSE+ExxSW) - comp*1/3*divS
# EzzS = 1/4*(EzzE+EzzW+EzzSE+EzzSW) - comp*1/3*divS
# GxzS = 1/dz*(uC-uS) + 1/dx*(vSE-vSW)
EdS  = Matrix([[ExxS], [EzzS], [GxzS]])
TS   = Dv*EdS
TxzS = TS[2].subs({D31:'D31S', D32:'D32S', D33:'D33S'})
#----------------------------------------#
# ExxN = 1/4*(ExxE+ExxW+ExxNE+ExxNW) - comp*1/3*divN
# EzzN = 1/4*(EzzE+EzzW+EzzNE+EzzNW) - comp*1/3*divN
# GxzN = 1/dz*(uN-uC) + 1/dx*(vNE-vNW)
EdN  = Matrix([[ExxN], [EzzN], [GxzN]])
TN   = Dv*EdN
TxzN = TN[2].subs({D31:'D31N', D32:'D32N', D33:'D33N'})
#----------------------------------------#
Fx = 1/dx*(TxxE - TxxW) + 1/dz*(TxzN - TxzS)  
Fx = -Fx
for i in range(len(dofs)):
    cUc = Fx.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cUc) + ';')

uW = -(0.25*D31N*inW*((1.0/3.0)*comp/dx - 1/dx) - 0.25*D31S*inW*((1.0/3.0)*comp/dx - 1/dx) + 0.083333333333333329*D32N*comp*inW/dx - 0.083333333333333329*D32S*comp*inW/dx)/dz - (-D11W*inW*((1.0/3.0)*comp/dx - 1/dx) - 1.0/3.0*D12W*comp*inW/dx - D13W*(-0.25*inNWu/dz + 0.25*inSWu/dz))/dx;
uC = -(D31N*(0.25*inE*((1.0/3.0)*comp/dx - 1/dx) + 0.25*inW*(-1.0/3.0*comp/dx + 1.0/dx)) - D31S*(0.25*inE*((1.0/3.0)*comp/dx - 1/dx) + 0.25*inW*(-1.0/3.0*comp/dx + 1.0/dx)) + D32N*(0.083333333333333329*comp*inE/dx - 0.083333333333333329*comp*inW/dx) - D32S*(0.083333333333333329*comp*inE/dx - 0.083333333333333329*comp*inW/dx) - D33N*inNu/dz - D33S*inSu/dz)/dz - (D11E*inE*((1.0/3.0)*comp/dx - 1/dx) - D11W*inW*(-1.0/3.0*comp/dx + 1.0/dx) + (1.0/3.0)*D12E*comp*inE/dx + (1.0/3.0)*D12W*comp*inW/dx + D13E*(-0.25*inNu/dz + 0.25*inSu/dz) - D13W*(-0.25*inNu/dz + 0.25*inSu/dz))/dx;
uE = -(0.25*D31N*inE*(-1.0/3.0*comp/dx + 1.0/dx) - 0.25*D31S*inE*(-1.0/3.0*comp/dx + 1.0/dx) - 0.083333333333333329*D32N*comp*inE/dx + 

In [17]:
# Compressibility switch
comp = symbols('comp')

# Stencil z
vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE = symbols('vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE')
dofs = Matrix([vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE])

# Stencil extension
vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE = symbols('vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE')

dofs_ext = Matrix([vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE])
dofs = np.append(dofs,dofs_ext)

inS,inN,inWu,inWv,inEu,inEv = symbols('inS,inN,inWu,inWv,inEu,inEv')

inSE,inNE,inSW,inNW = symbols('inSE,inNE,inSW,inNW')

# GxzSW = 1/dz*(uSW-uSSW) + 1/dx*(vS-vSW)
# GxzSE = 1/dz*(uSSE-uSE) + 1/dx*(vSE-vS)
# GxzW  = 1/dz*(uNW-uSW)  + 1/dx*(vC-vW)
# GxzE  = 1/dz*(uNE-uSE)  + 1/dx*(vE-vC)
# GxzNW = 1/dz*(uNW-uNNW) + 1/dx*(vN-vNW)
# GxzNE = 1/dz*(uNNE-uNE) + 1/dx*(vNE-vN)

# EzzS  = 1/dz*(vC-vS);
# EzzN  = 1/dz*(vN-vC);
# EzzSW = 1/dz*(vW-vSW);
# EzzNW = 1/dz*(vNW-vW);
# EzzSE = 1/dz*(vE-vSE);
# EzzNE = 1/dz*(vNE-vE)

# ExxS  = 1/dx*(uSE-uSW);
# ExxN  = 1/dx*(uNE-uNW);
# ExxSW = 1/dx*(uSW-uSWW);
# ExxNW = 1/dx*(uNW-uNWW);
# ExxSE = 1/dx*(uSEE-uSE);
# ExxNE = 1/dx*(uNEE-uNE);

# divS  = ExxS + EzzN
# divN  = ExxN + EzzN
# divSW = ExxSW + EzzSW;
# divNW = ExxNW + EzzNW;
# divSE = ExxSE + EzzSE;
# divNE = ExxNE + EzzNE;
# divW  = 1/4*(divS + divN + divSW + divNW) 
# divE  = 1/4*(divS + divN + divSE + divNE)

# Divergences
divS      = (uSE-uSW)/dx + (vC -vS )/dz
divN      = (uNE-uNW)/dx + (vN -vC )/dz
divSW     = (uSW-uSWW)/dx + (vW -vSW )/dz
divSE     = (uSEE-uSE)/dx + (vE -vSE )/dz
divNW     = (uNW-uNWW)/dx + (vNW -vW )/dz
divNE     = (uNEE-uNE)/dx + (vNE -vE )/dz
# Deviatoric normal strain
ExxS      = inS*((uSE-uSW)/dx - comp*1/3*divS)
EzzS      = inS*((vC -vS )/dz - comp*1/3*divS)
ExxN      = inN*((uNE-uNW)/dx - comp*1/3*divN)
EzzN      = inN*((vN -vC )/dz - comp*1/3*divN)
# Shear strain
GxzE      = inEu*(uNE-uSE)/dz + inEv*(vE - vC )/dx
GxzW      = inWu*(uNW-uSW)/dz + inWv*(vC - vW )/dx
# Additional missing stress tensor components via interpolation
GxzSW     = inSWu*(uSW  - uSSW)/dz + inSWv*(vS  - vSW)/dx
GxzSE     = inSEu*(uSE  - uSSE)/dz + inSEv*(vSE - vS )/dx
GxzNW     = inNWu*(uNNW - uNW )/dz + inNWv*(vN  - vNW)/dx
GxzNE     = inNEu*(uNNE - uNE )/dz + inNEv*(vNE - vN )/dx
ExxNE     = inNE*((uNEE-uNE)/dx - comp*1/3*(divNE))
ExxNW     = inNW*((uNW-uNWW)/dx - comp*1/3*(divNW))
ExxSE     = inSE*((uSEE-uSE)/dx - comp*1/3*(divSE))
ExxSW     = inSW*((uSW-uSWW)/dx - comp*1/3*(divSW))
EzzNE     = inNE*((vNE -vE )/dz - comp*1/3*(divNE))
EzzNW     = inNW*((vNW -vW )/dz - comp*1/3*(divNW))
EzzSE     = inSE*((vE -vSE )/dz - comp*1/3*(divSE))
EzzSW     = inSW*((vW -vSW )/dz - comp*1/3*(divSW))
# INV 0
GxzN      = 1/4*( GxzW + GxzE + GxzNW + GxzNE )
GxzS      = 1/4*( GxzW + GxzE + GxzSW + GxzSE )
ExxE      = 1/4*( ExxS + ExxN + ExxNE + ExxSE )
ExxW      = 1/4*( ExxS + ExxN + ExxNW + ExxSW )
EzzE      = 1/4*( EzzS + EzzN + EzzNE + EzzSE )
EzzW      = 1/4*( EzzS + EzzN + EzzNW + EzzSW )


#----------------------------------------#
# ExxS = 1/dx*(uSE-uSW) - comp*1/3*divS
# EzzS = 1/dz*( vC- vS) - comp*1/3*divS
# GxzS = 1/4*(GxzW+GxzE+GxzSW+GxzSE)
EdS  = Matrix([[ExxS], [EzzS], [GxzS]])
TS   = Dv*EdS
TzzS = TS[1].subs({D21:'D21S', D22:'D22S', D23:'D23S'})
#----------------------------------------#
# ExxN = 1/dx*(uNE-uNW) - comp*1/3*divN
# EzzN = 1/dz*( vN- vC) - comp*1/3*divN
# GxzN = 1/4*(GxzW+GxzE+GxzNW+GxzNE)
EdN  = Matrix([[ExxN], [EzzN], [GxzN]])
TN   = Dv*EdN
TzzN = TN[1].subs({D21:'D21N', D22:'D22N', D23:'D23N'})
#----------------------------------------#
# ExxW = 1/4*(ExxS+ExxN+ExxSW+ExxNW) - comp*1/3*divW
# EzzW = 1/4*(EzzS+EzzN+EzzSW+EzzNW) - comp*1/3*divW
# GxzW = 1/dz*(uNW-uSW) + 1/dx*(vC-vW)
EdW  = Matrix([[ExxW], [EzzW], [GxzW]])
TW   = Dv*EdW
TxzW = TW[2].subs({D31:'D31W', D32:'D32W', D33:'D33W'})
# #----------------------------------------#
# ExxE = 1/4*(ExxS+ExxN+ExxSE+ExxNE) - comp*1/3*divE
# EzzE = 1/4*(EzzS+EzzN+EzzSE+EzzNE) - comp*1/3*divE
# GxzE = 1/dz*(uNE-uSE) + 1/dx*(vE-vC)
EdE  = Matrix([[ExxE], [EzzE], [GxzE]])
TE   = Dv*EdE
TxzE = TE[2].subs({D31:'D31E', D32:'D32E', D33:'D33E'})
#----------------------------------------#
Fz = 1/dz*(TzzN - TzzS) + 1/dx*(TxzE - TxzW)  
Fz = -Fz
for i in range(len(dofs)):
    cVc = Fz.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cVc) + ';') #print(cUc)

vW = -(-0.25*D23N*inWv/dx + 0.25*D23S*inWv/dx)/dz - (-D31W*(0.083333333333333329*comp*inNW/dz - 0.083333333333333329*comp*inSW/dz) - D32W*(0.25*inNW*((1.0/3.0)*comp/dz - 1/dz) + 0.25*inSW*(-1.0/3.0*comp/dz + 1.0/dz)) + D33W*inWv/dx)/dx;
vC = -((1.0/3.0)*D21N*comp*inN/dz + (1.0/3.0)*D21S*comp*inS/dz + D22N*inN*((1.0/3.0)*comp/dz - 1/dz) - D22S*inS*(-1.0/3.0*comp/dz + 1.0/dz) + D23N*(-0.25*inEv/dx + 0.25*inWv/dx) - D23S*(-0.25*inEv/dx + 0.25*inWv/dx))/dz - (D31E*(0.083333333333333329*comp*inN/dz - 0.083333333333333329*comp*inS/dz) - D31W*(0.083333333333333329*comp*inN/dz - 0.083333333333333329*comp*inS/dz) + D32E*(0.25*inN*((1.0/3.0)*comp/dz - 1/dz) + 0.25*inS*(-1.0/3.0*comp/dz + 1.0/dz)) - D32W*(0.25*inN*((1.0/3.0)*comp/dz - 1/dz) + 0.25*inS*(-1.0/3.0*comp/dz + 1.0/dz)) - D33E*inEv/dx - D33W*inWv/dx)/dx;
vE = -(0.25*D23N*inEv/dx - 0.25*D23S*inEv/dx)/dz - (D31E*(0.083333333333333329*comp*inNE/dz - 0.083333333333333329*comp*inSE/dz) + D32E*(0.25*inNE*((1.0/3.0)*comp/dz - 1/dz) + 0.25*inSE